In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Import mlc_data.csv
mlc_df = pd.read_csv("../data/mlc_data.csv")
mlc_df.head()

,If,Country Code,Registrant Code,Year of Reference,Usage Period,Streaming Platform (DSP),Streams,Recording Artist,Recording Label,Recording Title,Release Title,Recording Duration (Seconds),"Songwriter's Listed (1 = ""Yes"", 0 = ""No"")"
0,USUG12101043,US,UG1,21.0,NaN,AudioMack,"10,175",Future,NaN,FUTURE FT DEJ LOAF HEY THERE PROD BY DDS,#unknown#,181,0
1,USRC12100543,US,RC1,21.0,NaN,SoundCloud,"8,597",LUCKY3RD,LUCKY3RD,Keep It Cool LUCKY3RD,Keep It Cool LUCKY3RD,133,0
2,USSM12102263,US,SM1,21.0,NaN,SoundCloud,"261,280",LUCKY3RD,LUCKY3RD,Life Goes On LUCKY3RD,Life Goes On LUCKY3RD,171,0
3,USLD91731547,US,LD9,17.0,2/1/2021,Trebel,5,Bachata & Merengue Mix,Orchard,No dudes de mi- Merengue & Bachata Mix,Mega Mix 2010,1250,0
4,USAT22007048,US,AT2,20.0,NaN,AudioMack,"62,105",Foolio,NaN,WHEN I SEE YOU REMIX,#unknown#,187,0


In [4]:
# Abi: Rename column If to ISRC -- ORIGINAL 'If ' column HAS A SPACE after it!!
mlc_df.columns = ['ISRC', 'Country', 'Registrant', 'Release_Year', 'Period', 'DSP', 'Streams', 'Artist', 'Label', 'Song', 'Album', 'Duration', 'Songwriter?']
mlc_df.head()

,ISRC,Country,Registrant,Release_Year,Period,DSP,Streams,Artist,Label,Song,Album,Duration,Songwriter?
0,USUG12101043,US,UG1,21.0,NaN,AudioMack,"10,175",Future,NaN,FUTURE FT DEJ LOAF HEY THERE PROD BY DDS,#unknown#,181,0
1,USRC12100543,US,RC1,21.0,NaN,SoundCloud,"8,597",LUCKY3RD,LUCKY3RD,Keep It Cool LUCKY3RD,Keep It Cool LUCKY3RD,133,0
2,USSM12102263,US,SM1,21.0,NaN,SoundCloud,"261,280",LUCKY3RD,LUCKY3RD,Life Goes On LUCKY3RD,Life Goes On LUCKY3RD,171,0
3,USLD91731547,US,LD9,17.0,2/1/2021,Trebel,5,Bachata & Merengue Mix,Orchard,No dudes de mi- Merengue & Bachata Mix,Mega Mix 2010,1250,0
4,USAT22007048,US,AT2,20.0,NaN,AudioMack,"62,105",Foolio,NaN,WHEN I SEE YOU REMIX,#unknown#,187,0


In [20]:
mlc_df.tail()

,ISRC,Country,Registrant,Release_Year,Period,DSP,Streams,Artist,Label,Song,Album,Duration,Songwriter?
99995,QZK6F2019397,QZ,K6F,20.0,3/1/2021,Spotify,"28,549",Myuk,Sony Music Labels Inc.,魔法 - Anime Size,魔法 (Anime Size),89,1
99996,QZK6F2019397,QZ,K6F,20.0,3/1/2021,Spotify,"28,941",yama,Sony Music Labels Inc.,麻痺,麻痺,199,1
99997,SE6HN1926755,SE,6HN,19.0,3/1/2021,Spotify,"28,941",yama,Sony Music Labels Inc.,麻痺,麻痺,199,1
99998,SEYOK1669274,SE,YOK,16.0,3/1/2021,Apple,"8,970",yama,Sony Music Labels Inc.,麻痺,麻痺 - Single,198,1
99999,QZK6F2019397,QZ,K6F,20.0,2/1/2021,Spotify,"11,245",Lexie Liu,泥鞋（北京）文化传媒有限公司,黑洞 Metropolis,无限意识 Meta Ego,259,1


In [5]:
# Inspecting the dataframe made from mlc_data.csv
print(mlc_df.shape)
print(mlc_df.dtypes)

(100000, 13)
ISRC             object
Country          object
Registrant       object
Release_Year    float64
Period           object
DSP              object
Streams          object
Artist           object
Label            object
Song             object
Album            object
Duration          int64
Songwriter?       int64
dtype: object


In [6]:
# Most popular streaming platform by number of records in the dataframe
mlc_df['DSP'].value_counts()

Spotify               32268
Apple                 22200
Amazon                14438
Pandora               13777
Tidal                  3521
YouTube                2752
SoundCloud             2122
GTL                    1090
Melodyv                 829
Trebel                  817
iHeart Radio            707
AudioMack               550
NugsNet                 316
LiveXLive               200
Qoboz                   104
Midwest Tape            102
Deezer                   51
Anghami                  46
Sonos                    23
Recisio                  22
Smithsonian              21
Ultimate Guitar          19
PowerMusic                8
Wolfgangs                 4
Fan Label                 4
MixCloud                  4
Pacemaker                 3
Classical Archives        2
MonkingMe                 1
Name: DSP, dtype: int64

In [ ]:
# Abi:
raw_df["Streams"] = raw_df["Streams"].str.replace(",", "")
raw_df["Streams"] = raw_df["Streams"].str.replace(" ", "")
raw_df[["Streams"]] = raw_df[["Streams"]].apply(pd.to_numeric)

In [7]:
# Patrick: Most popular streaming platform by number of records in the dataframe with US country code
domestic = mlc_df.loc[mlc_df['Country']== 'US']
domestic['DSP'].value_counts()

Spotify               13614
Apple                  9389
Amazon                 5919
Pandora                5646
Tidal                  1425
YouTube                1095
SoundCloud              857
GTL                     438
Melodyv                 342
Trebel                  333
iHeart Radio            284
AudioMack               234
NugsNet                 142
LiveXLive                84
Qoboz                    55
Midwest Tape             39
Deezer                   29
Anghami                  24
Recisio                  13
Smithsonian              13
Ultimate Guitar          10
Sonos                     9
Wolfgangs                 2
PowerMusic                2
Pacemaker                 1
MixCloud                  1
Fan Label                 1
Classical Archives        1
Name: DSP, dtype: int64

In [8]:
# Patrick: Most popular streaming platform by number of records in the dataframe with non-US (foreign) country code
foreign = mlc_df.loc[mlc_df['Country']!= 'US']
foreign['DSP'].value_counts()

Spotify               18654
Apple                 12811
Amazon                 8519
Pandora                8131
Tidal                  2096
YouTube                1657
SoundCloud             1265
GTL                     652
Melodyv                 487
Trebel                  484
iHeart Radio            423
AudioMack               316
NugsNet                 174
LiveXLive               116
Midwest Tape             63
Qoboz                    49
Anghami                  22
Deezer                   22
Sonos                    14
Recisio                   9
Ultimate Guitar           9
Smithsonian               8
PowerMusic                6
Fan Label                 3
MixCloud                  3
Wolfgangs                 2
Pacemaker                 2
MonkingMe                 1
Classical Archives        1
Name: DSP, dtype: int64

In [9]:
# Most popular streams of all
mlc_df.sort_values('Streams')

,ISRC,Country,Registrant,Release_Year,Period,DSP,Streams,Artist,Label,Song,Album,Duration,Songwriter?
21352,QZDA41861694,QZ,DA4,18.0,3/1/2021,GTL,1,DJ Just Dizle;DJ Spinna,Big Lynden Radio,"Dearly Departed, Pt. 2 (The Rip Mixtape)",Dearly Departed (The Rip Mixtape) (Dj Just Diz...,5225,0
741,USPJT2100004,US,PJT,21.0,3/1/2021,GTL,1,Malcolm X,Master Classics,1964 NYC Radio Interview,All Time Greatest Speeches,14553,0
742,QMCE32100212,QM,CE3,21.0,3/1/2021,GTL,1,Malcolm X,Master Classics,"1964 Oauu Rally, Audubon Ballroom","All Time Greatest Speeches, Vol. 2",4216,0
743,USBAL1600001,US,BAL,16.0,3/1/2021,GTL,1,Malcolm X,Master Classics,1964 Worldwide Revolution,All Time Greatest Speeches,4203,0
744,JPVI01817011,JP,VI0,18.0,3/1/2021,GTL,1,Malcolm X,Master Classics,1964 Worldwide Revolution,All Time Greatest Speeches,4203,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14146,US5261923213,US,526,19.0,4/1/2021,Spotify,"998,139","BROCKHAMPTON,Danny Brown",Question Everything/RCA Records,BUZZCUT (feat. Danny Brown),BUZZCUT (feat. Danny Brown),202,1
14148,USDXS1802700,US,DXS,18.0,5/1/2021,Spotify,"998,139","BROCKHAMPTON,Danny Brown",Question Everything/RCA Records,BUZZCUT (feat. Danny Brown),BUZZCUT (feat. Danny Brown),202,1
15990,USAT21804343,US,AT2,18.0,3/1/2021,Amazon,999,Leo,Leo,Chandelier - Metal Cover,Leo Metal Covers Volume 2,206,0
41418,USUM71909791,US,UM7,19.0,3/1/2021,Amazon,999,Punk Goes,Fearless Records,I Want It That Way,Punk Goes Pop,211,0


In [10]:
# Rudy: More info
mlc_df.describe()

,Release_Year,Duration,Songwriter?
count,98239.000000,100000.000000,100000.000000
mean,19.671933,1016.893690,0.541610
std,12.638870,15565.692133,0.498268
min,0.000000,0.000000,0.000000
25%,17.000000,149.000000,0.000000
50%,19.000000,190.000000,1.000000
75%,20.000000,236.000000,1.000000
max,99.000000,818738.000000,1.000000


## Finding the ratio (=0-1) of songwriter info provided by DSP

In [19]:
# Exploring Songwriter? values -- there are more recordings that have songwriter info provided than not
mlc_df['Songwriter?'].value_counts()

1    54161
0    45839
Name: Songwriter?, dtype: int64

In [24]:
# Make a df with just the DSP and Songwriter? columns: sw_df (sw=songwriter)
sw_df = mlc_df[['DSP', 'Songwriter?']]
sw_df.head()

,DSP,Songwriter?
0,AudioMack,0
1,SoundCloud,0
2,SoundCloud,0
3,Trebel,0
4,AudioMack,0


In [26]:
sw_df['DSP']
spotify_df = sw_df[sw_df['DSP'].isin(['Spotify'])]
spotify_df.head()

,DSP,Songwriter?
5,Spotify,0
8,Spotify,1
9,Spotify,1
11,Spotify,1
12,Spotify,1


In [27]:
spotify_df['Songwriter?'].value_counts()

1    27569
0     4699
Name: Songwriter?, dtype: int64

## Doing this for each DSP will be tedious, maybe try a for loop that iterates over each unique value in the DSP column?
https://stackoverflow.com/questions/36684013/extract-column-value-based-on-another-column-pandas-dataframe

In [29]:
# All unique DSPs in mlc_df (taken from sw_df)
sw_df['DSP'].value_counts()

Spotify               32268
Apple                 22200
Amazon                14438
Pandora               13777
Tidal                  3521
YouTube                2752
SoundCloud             2122
GTL                    1090
Melodyv                 829
Trebel                  817
iHeart Radio            707
AudioMack               550
NugsNet                 316
LiveXLive               200
Qoboz                   104
Midwest Tape            102
Deezer                   51
Anghami                  46
Sonos                    23
Recisio                  22
Smithsonian              21
Ultimate Guitar          19
PowerMusic                8
Wolfgangs                 4
Fan Label                 4
MixCloud                  4
Pacemaker                 3
Classical Archives        2
MonkingMe                 1
Name: DSP, dtype: int64

In [33]:
# How many records have songwriter info by DSP?
yes_sw = sw_df.loc[sw_df['Songwriter?'] == 1, 'DSP']
yes_sw.value_counts()

Spotify               27569
Apple                 17949
Amazon                 3779
Pandora                3373
Melodyv                  95
Qoboz                    94
Deezer                   44
GTL                      31
LiveXLive                14
Tidal                    13
Recisio                  12
Classical Archives        2
Name: DSP, dtype: int64

In [34]:
# How many records do NOT have songwriter info by DSP?
no_sw = sw_df.loc[sw_df['Songwriter?'] == 0, 'DSP']
no_sw.value_counts()

Amazon             10659
Pandora            10404
Spotify             4699
Apple               4251
Tidal               3508
YouTube             2752
SoundCloud          2122
GTL                 1059
Trebel               817
Melodyv              734
iHeart Radio         707
AudioMack            550
NugsNet              316
LiveXLive            186
Midwest Tape         102
Anghami               46
Sonos                 23
Smithsonian           21
Ultimate Guitar       19
Recisio               10
Qoboz                 10
PowerMusic             8
Deezer                 7
Wolfgangs              4
Fan Label              4
MixCloud               4
Pacemaker              3
MonkingMe              1
Name: DSP, dtype: int64